# Running a SCF

In [1]:
import Auto_baby

In [2]:
print(Auto_baby)

<module 'Auto_baby' from '/home/student/zylu/fireworks/Auto_baby.py'>


In [6]:
silicon_experiment = Auto_baby.Experiment(material_dir='/home/student/zylu/silicon_demo')

In [7]:
silicon_experiment.ensure_directories()

All calculations are organized in directories under /home/student/zylu/silicon_demo based on their names.


In [8]:
silicon_experiment.ensure_required_files()

All required calculation inputs and job scripts are present where needed.


In [9]:
silicon_experiment.scf_calculation()

Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING


KeyboardInterrupt: 

In [10]:
silicon_experiment.dos()

Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING


KeyboardInterrupt: 

In [11]:
silicon_experiment.band_structure()

Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING


KeyboardInterrupt: 

# Automation with FireWorks

## Download MP files from their offical repo and save them to POSCAR

In [7]:
from mp_api.client import MPRester

with MPRester("5ypNyStQsaDUA2neNLD17P8v2KNB241U") as mpr:
    
    structure = mpr.get_structure_by_material_id("mp-149") 
    structure.to(fmt="poscar")  
    structure.to_file("/home/student/zylu/demo/1_scf/POSCAR")  
    print(structure)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.849278   3.849279   3.849278
angles:  60.000012  60.000003  60.000011
pbc   :       True       True       True
Sites (2)
  #  SP        a      b      c    magmom
---  ----  -----  -----  -----  --------
  0  Si    0.875  0.875  0.875        -0
  1  Si    0.125  0.125  0.125        -0


/home/student/Programs/anaconda3/envs/fireworks/lib/python3.14/site-packages/pymatgen/io/vasp/inputs.py:619: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


## WorkFlows

In [ ]:
from fireworks import Firework, LaunchPad, FileTransferTask, PyTask, Workflow
from fireworks.core.rocket_launcher import rapidfire
import Auto_baby

def Final_Calculation(material_path:str = ''):
    if not material_path: raise ValueError("material_path must be provided.")


    launchpad = LaunchPad()
    launchpad.reset('', require_password=False)
    
    experiment = Auto_baby.Experiment(material_dir=material_path)
    experiment.ensure_directories()
    experiment.ensure_required_files()
    
    _submit_SCF_Task_ = PyTask(func = experiment.scf_calculation())

    _transfer_CONTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CONTCAR', 
                                                       'dest': f'{material_path}/2_dos/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                       'dest': f'{material_path}/2_dos/POTCAR'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                  'dest': f'{material_path}/2_dos/job.sh'}], 'mode': 'copy'})
    _transfer_WAVECAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/WAVECAR', 
                                                       'dest': f'{material_path}/2_dos/WAVECAR'}], 'mode': 'copy'})
    

    _submit_DOS_Task_ = PyTask(func = experiment.dos())



    _transfer_CONTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CONTCAR', 
                                                    'dest': f'{material_path}/3_bs/POSCAR'}], 'mode': 'copy'})
    _transfer_POTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                    'dest': f'{material_path}/3_bs/POTCAR'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                    'dest': f'{material_path}/3_bs/job.sh'}], 'mode': 'copy'})
    _transfer_CHGCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CHGCAR', 
                                                    'dest': f'{material_path}/3_bs/CHGCAR'}], 'mode': 'copy'})
    

    _submit_BS_Task_ = PyTask(func = experiment.band_structure())



    scf_calculation = Firework(
        tasks=[_submit_SCF_Task_],
        name="SCF Calculation" 
    )

    DOS_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_DOS, _transfer_POTCAR_of_SCF_to_DOS, _transfer_JOB_of_SCF_to_DOS, _transfer_WAVECAR_of_SCF_to_DOS,_submit_DOS_Task_],
        name="DOS Calculation",
        parents=[scf_calculation]
    )

    BAND_structure_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_BS, _transfer_POTCAR_of_SCF_to_BS, _transfer_JOB_of_SCF_to_BS, _transfer_CHGCAR_of_SCF_to_BS,_submit_BS_Task_],
        name="Band Structure Calculation",
        parents=[scf_calculation]
    )

    workflow_order = [scf_calculation, DOS_calculation, BAND_structure_calculation]

    wf = Workflow(workflow_order, {scf_calculation.fw_id: [DOS_calculation.fw_id, BAND_structure_calculation.fw_id]})
    launchpad.add_wf(wf)

    rapidfire(launchpad)

In [14]:
Final_Calculation(material_path='/home/student/zylu/silicon_demo')

2025-11-13 20:51:25,221 INFO Performing db tune-up
2025-11-13 20:51:25,231 INFO LaunchPad was RESET.
Current SLURM job state: UNKNOWN
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current SLURM job state: PENDING
Current 

Traceback (most recent call last):
  File "/home/student/Programs/anaconda3/envs/fireworks/lib/python3.14/site-packages/fireworks/core/rocket.py", line 260, in run
    m_action = t.run_task(my_spec)
  File "/home/student/Programs/anaconda3/envs/fireworks/lib/python3.14/site-packages/fireworks/user_objects/firetasks/script_task.py", line 163, in run_task
    toks = self["func"].rsplit(".", 1)
           ^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'rsplit'


## PostProcessing (py4vasp)

In [13]:
import py4vasp
mycalc = py4vasp.Calculation.from_file("/home/student/zylu/demo/2_dos/vaspout.h5")

mycalc.dos.plot()

mycalc.dos.to_image("dos_plot.png")


In [12]:
import py4vasp
mycalc = py4vasp.Calculation.from_file("/home/student/zylu/demo/3_bs/vaspout.h5")

mycalc.band.plot()